In [1]:
import os, sys
import argparse
import time
from os.path import exists
import collections
from typing import Iterable
import pickle                                                                                
from collections import Counter

import scanpy as sc
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

# Preprocess input data 

Here we show how we preprocessed the datasets we used in our manuscript, except for the Tabula Muris datasets which we already covered in our examples (see [getExample.py](https://github.com/doraadong/UNIFAN/blob/main/tutorails/getExample.py) for details). Other than the pacakges imported above, we also use [mygene](https://pypi.org/project/mygene/) package to convert ENSEMBL IDs to gene symbols for the HuBMAP datasets. 

**Table of Content**

1. [Preprocess pbmc28K](#1)
2. [Preprocess pbmc68K](#2)
3. [Preprocess Atlas lung](#3)
4. [Preprocess HuBMAP datasets](#4)

In [3]:
cell_cutoffs = {'lung': 500, 'lymph_node':200, 'spleen':200, 'thymus':200, 'pbmc68k':200, 'pbmc28k':200}
gene_cutoffs = {'lung': 5, 'lymph_node':3, 'spleen':3, 'thymus':3, 'pbmc68k':3, 'pbmc28k':3}

In [4]:
save_raw = False
save_processed = True

topk = 2000
select_genes = False

if select_genes: 
    condition = f"_top{topk}"
else:
    condition = f""

## 1. Preprocess pbmc28k
<a id=1></a>

In [6]:
tissue = "pbmc28k"
data_name = "pbmc"

parent_folder = f"../input/{data_name}"
labels = pd.read_csv(f"{parent_folder}/{tissue}/barcodes_to_cell_types.tsv", sep="\t")

barcodes = [i.split('_')[0] for i in labels['barcode']]
lanes = [i.split('_')[1] for i in labels['barcode']]
labels['code'] = barcodes
labels['lane'] = lanes

In [7]:
full_data = None
pre_genes = None 

for i in range(1, 9):
    cur_lane = f"lane_{i}"
    
    parent_folder = f"../input/{data_name}"
    adata = sc.read_10x_mtx(f"{parent_folder}/{tissue}/{cur_lane}/", var_names='gene_symbols', cache=True)          
    adata.var_names_make_unique() 
    
    print(adata)
    
    # get observation idx
    _codes = [i.split('-')[0] for i in adata.obs.index.values]
    # all barcodes having labels are in the expression file 
    assert len(set(labels[labels['lane'] == f"lane{i}"].code.values) - set(_codes)) == 0

    _others = [i.split('-')[1] for i in adata.obs.index.values]
    assert (np.array(_others) == '1').all()
    
    # get labels 
    cur_labels = labels[labels['lane'] == f"lane{i}"].copy().reset_index()
    cur_labels = cur_labels[['code', 'cell_type']]

    df_codes = pd.DataFrame(_codes)
    df_codes.columns = ['code']
    df_codes = df_codes.merge(cur_labels, left_on = 'code', right_on = 'code', how = 'left')

    # same order as in adata
    assert (df_codes['code'].values == np.array(_codes)).all()
    
    adata.obs['label'] = df_codes['cell_type'].values
    
    if full_data is not None:
        full_data = full_data.concatenate(adata)
    else:
        full_data = adata
    
    # check if gene id same for all data
    cur_genes = adata.var['gene_ids'].values
    if pre_genes is not None:
        assert (pre_genes == cur_genes).all()
    pre_genes = cur_genes

AnnData object with n_obs × n_vars = 4309 × 32738
    var: 'gene_ids'
AnnData object with n_obs × n_vars = 3844 × 32738
    var: 'gene_ids'
AnnData object with n_obs × n_vars = 4998 × 32738
    var: 'gene_ids'
AnnData object with n_obs × n_vars = 3392 × 32738
    var: 'gene_ids'
AnnData object with n_obs × n_vars = 2868 × 32738
    var: 'gene_ids'
AnnData object with n_obs × n_vars = 3685 × 32738
    var: 'gene_ids'
AnnData object with n_obs × n_vars = 3198 × 32738
    var: 'gene_ids'
AnnData object with n_obs × n_vars = 2561 × 32738
    var: 'gene_ids'


In [8]:
# keep only those with labels
full_data = full_data[~full_data.obs.label.isna()]

In [9]:
full_data

View of AnnData object with n_obs × n_vars = 25185 × 32738
    obs: 'label', 'batch'
    var: 'gene_ids'

In [10]:
# filtering 
sc.pp.filter_cells(full_data, min_genes=cell_cutoffs[tissue])
sc.pp.filter_genes(full_data, min_cells=gene_cutoffs[tissue])

if save_raw:
    filename = f"{tissue}_raw{condition}.h5ad"
    full_data.write_h5ad(os.path.join(parent_folder, filename))

if save_processed:
    # selecting genes using Seurat v3 method 
    sc.pp.highly_variable_genes(full_data, n_top_genes=topk, flavor='seurat_v3')
    if select_genes:
        full_data = full_data[:, full_data.var.highly_variable]

    # normalize 
    sc.pp.normalize_total(full_data, target_sum=1e4)
    sc.pp.log1p(full_data)

    # adata.raw = adata
    sc.pp.scale(full_data, max_value=10)

    # save 
    try:
        full_data.__dict__['_raw'].__dict__['_var'] = full_data.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
    except AttributeError:
        pass 

    filename = f"{tissue}_processed{condition}.h5ad"
    full_data.write_h5ad(os.path.join(parent_folder, filename))

Trying to set attribute `.obs` of view, copying.
... storing 'label' as categorical


## 2. Preprocessing pbmc68k
<a id=2></a>

In [6]:
tissue = "pbmc68k"
data_name = "pbmc"

parent_folder = f"../input/{data_name}"
full_data = sc.read_10x_mtx(f"{parent_folder}/{tissue}/filtered_matrices_mex/hg19/", var_names='gene_symbols',cache=True)          
full_data.var_names_make_unique() 

# get labels 
labels = pd.read_csv(f"{parent_folder}/{tissue}/zheng17-cell-labels.txt", sep = "\t")

# check if barcodes same
barcodes = pd.read_csv("../input/pbmc/pbmc68k/filtered_matrices_mex/hg19/barcodes.tsv", header = None)
assert (labels['barcode'].values == barcodes[0].values).all()

full_data.obs['label'] = labels['bulk_labels'].values

# filtering 
sc.pp.filter_cells(full_data, min_genes=cell_cutoffs[tissue]) 
sc.pp.filter_genes(full_data, min_cells=gene_cutoffs[tissue])

if save_raw:
    filename = f"{tissue}_raw{condition}.h5ad"
    full_data.write_h5ad(os.path.join(parent_folder, filename))
    
if save_processed:
    # selecting genes using Seurat v3 method 
    sc.pp.highly_variable_genes(full_data, n_top_genes=topk, flavor='seurat_v3')
    if select_genes:
        full_data = full_data[:, full_data.var.highly_variable]

    # normalie 
    sc.pp.normalize_total(full_data, target_sum=1e4)
    sc.pp.log1p(full_data)

    # adata.raw = adata
    sc.pp.scale(full_data, max_value=10)

    # save 
    try:
        full_data.__dict__['_raw'].__dict__['_var'] = full_data.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
    except AttributeError:
        pass 

    filename = f"{tissue}_processed{condition}.h5ad"
    full_data.write_h5ad(os.path.join(parent_folder, filename))

... storing 'label' as categorical


## 3. Preprocess Atlas lung
<a id=3></a>

Before running the following code, make sure you have done the following: 

1. Download the following two files from [GSE136831](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE136831)
    * GSE136831_AllCells.Samples.CellType.MetadataTable.txt
    * GSE136831_RawCounts_Sparse.mtx 
2. Take only samples where "Disease_Identity" == "Control"
3. Make a AnnData (i.e. lung.h5ad) using the raw counts as the expression and the meta data as the observation matrix. Make sure to include the following two columns in the observation matrix:
    * CellType_Category - coarse-grained label 
    * Manuscript_Identity - fine-grained label 

In [ ]:
tissue = "lung"

filename = f"{tissue}.h5ad"
parent_folder = "../input/ipf"

full_data = sc.read(os.path.join(parent_folder, filename), dtype='float64')

# filtering 
sc.pp.filter_cells(full_data, min_genes=cell_cutoffs[tissue]) 
sc.pp.filter_genes(full_data, min_cells=gene_cutoffs[tissue])

if save_raw:
    # filter using genes in the preprocessed data
    filename = f"{tissue}_processed{condition}.h5ad"
    proccessed_data = sc.read(os.path.join(parent_folder, filename), dtype='float64', backed="r")

    print(full_data.var.index.is_unique)
    cur_genes = list(full_data.var.index.values)

    idx_genes = [cur_genes.index(i) for i in proccessed_data.var.index.values]
    full_data = full_data[:, idx_genes]

    assert (full_data.var.index.values == proccessed_data.var.index.values).all()
    assert (full_data.obs.index.values == proccessed_data.obs.index.values).all()

    try:
        full_data.__dict__['_raw'].__dict__['_var'] = full_data.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
    except AttributeError:
        pass 


    filename = f"{tissue}_raw{condition}.h5ad"
    full_data.write_h5ad(os.path.join(parent_folder, filename))

if save_processed:

    # normalie 
    sc.pp.normalize_total(full_data, target_sum=1e4)
    sc.pp.log1p(full_data)

    # given the large number of genes for this data; filter to keep only highly dispersed genes 
    sc.pp.highly_variable_genes(full_data, min_mean=0, max_mean = 1000, min_disp=0.01)
    full_data = full_data[:, full_data.var.highly_variable]

    # adata.raw = adata
    sc.pp.scale(full_data, max_value=10)

    # save 
    try:
        full_data.__dict__['_raw'].__dict__['_var'] = full_data.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
    except AttributeError:
        pass 

    filename = f"{tissue}_processed{condition}.h5ad"
    full_data.write_h5ad(os.path.join(parent_folder, filename))

##### add highly variables genes selected by Seurat_v3 for lung

In [5]:
tissue = "lung"

filename = f"{tissue}.h5ad"
parent_folder = "../input/ipf"

full_data = sc.read(os.path.join(parent_folder, filename), dtype='float64')

# filtering 
sc.pp.filter_cells(full_data, min_genes=cell_cutoffs[tissue]) 
sc.pp.filter_genes(full_data, min_cells=gene_cutoffs[tissue])

# selecting genes using Seurat v3 method 
sc.pp.highly_variable_genes(full_data, n_top_genes=topk, flavor='seurat_v3')  
genes_variable = full_data.var.index[full_data.var.highly_variable]

In [33]:
filename = f"{tissue}_processed.h5ad"
temp = sc.read(os.path.join(parent_folder, filename), dtype='float64', backed = "r")

In [17]:
print(f"Number of Seurat selected genes not in dispersion-selected: {len(set(genes_variable) - set(temp.var.index))}")

Number of Seurat selected genes not in dispersion-selected: 100


In [20]:
highly_variable_seurat = np.repeat(0, temp.shape[1])

for i in range(temp.shape[1]):
    g = temp.var.index.values[i]
    if g in genes_variable:
        highly_variable_seurat[i] = 1
        
temp.var["highly_variable_seurat"] =  highly_variable_seurat.astype(bool)

In [32]:
filename = f"{tissue}_processed.h5ad"
temp.write_h5ad(os.path.join(parent_folder, filename))

## 4. Preprocess HuBMAP datasets
<a id=4></a>

For each tissue, we made the data by concatenating multiple datasets from HuBMAP. Please see [HuBMAP_datasets_ID.xlsx](https://github.com/doraadong/UNIFAN/blob/main/tutorails/HuBMAP_datasets_ID.xlsx) for the dataset IDs for the corresponding tissues. All datasets are preprocessed using standardized [HuBMAP scRNA-seq pipeline](https://github.com/hubmapconsortium/salmon-rnaseq).

In [6]:
for tissue in ["spleen", "thymus", "lymph_node"]:
    
    filename = f"{tissue}.h5ad"
    parent_folder = "../input/hubmap"

    full_data = sc.read(os.path.join(parent_folder, filename), dtype='float64')

    # filtering 
    sc.pp.filter_cells(full_data, min_genes=cell_cutoffs[tissue]) 
    sc.pp.filter_genes(full_data, min_cells=gene_cutoffs[tissue])
    
    if save_raw:     
        try:
            full_data.__dict__['_raw'].__dict__['_var'] = full_data.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
        except AttributeError:
            pass 
        
        filename = f"{tissue}_raw{condition}.h5ad"
        full_data.write_h5ad(os.path.join(parent_folder, filename))
            
    if save_processed:
        # selecting genes using Seurat v3 method 
        sc.pp.highly_variable_genes(full_data, n_top_genes=topk, flavor='seurat_v3')

        if select_genes:
            full_data = full_data[:, full_data.var.highly_variable]

        # normalie 
        sc.pp.normalize_total(full_data, target_sum=1e4)
        sc.pp.log1p(full_data)

        # adata.raw = adata
        sc.pp.scale(full_data, max_value=10)

        # convert ensemble to gene symbols 
        import mygene
        mg = mygene.MyGeneInfo()

        genes = [g.split('.')[0] for g in full_data.var.index.values]

        out = mg.querymany(genes, scopes='ensembl.gene', fields='symbol', species='human', returnall=True)


        # check when not selecting genes 
        for i in out['out']:
            if i['query'] == 'ENSG00000229425':
                print(i)

        for i in out['out']:
            if i['query'] == 'ENSG00000130723':
                print(i)

        en2symbol = {}
        for i in out['out']:
            if 'symbol' in i.keys():
                if i['query'] not in en2symbol.keys():
                    en2symbol[i['query']] = [i['symbol']]
                else:
                    en2symbol[i['query']].append(i['symbol'])
            else:
                en2symbol[i['query']] = [i['query']]

        symbols = []
        for g in genes:
            symbols.append(en2symbol[g][0])  # take the first one 

        full_data.var.index = symbols


        # save 
        try:
            full_data.__dict__['_raw'].__dict__['_var'] = full_data.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})
        except AttributeError:
            pass 

        filename = f"{tissue}_processed{condition}.h5ad"
        full_data.write_h5ad(os.path.join(parent_folder, filename))

... storing 'tissue' as categorical
... storing 'predicted.id' as categorical
... storing 'celltype' as categorical
... storing 'tissue' as categorical
... storing 'predicted.id' as categorical
... storing 'celltype' as categorical


True
